In [1]:
#fixing code from submission

In [12]:
#importing packages
from bs4 import BeautifulSoup 
import requests  
from datetime import datetime as dt
import math

#getting inputs from user
search_term = input("Enter a search term:")
min_severity = float(input("Type in a minimum severity level (1-10):"))
start = input("Type in a start publish date (MM-DD-YYYY):")
end = input("Type in a end publish date (MM-DD-YYYY):")

#Formatting date inputs
format_str = '%m-%d-%Y'
start_date = dt.strptime(start, format_str)
end_date = dt.strptime(end, format_str)

#URL for web scraping with term input into the url string
url = 'https://nvd.nist.gov/vuln/search/results?form_type=Basic&results_type=overview&query='+search_term+'&search_type=all&startIndex=0'

# Request content from web page and geting the soup from the response
response = requests.get(url)
content = response.content
soup = BeautifulSoup(content, 'lxml')

#getting the length of the page by taking the page lengths and dividing it by 20 (how many rows in each page)
searchLength = soup.find('strong', {"data-testid": "vuln-matching-records-count"})

#taking total rows found and dividing by 20 to find the page count and rounding it down to nearest int
total = searchLength.text
pages = math.floor(int(total) / 20)

#print amount of data points and pages total
print("\n")
print("The search returned " + total + " results. Use the following criteria to refine your search.")
print("There are " + str(pages) + " pages \n")
print("Genterating Results, please wait...\n")
print("\n")

#create arrays for populating data after refining scrape

#vulnerability IDs
vul_IDs =[]
#severity levels
severities = []
#publish dates 
publish_dates = []
#the urls of individual vulnerbility description pages
urls = []

#loop through pages to populate the arrays
for page in range(pages):
    
    #go through each page and add find rows of the tables
    url = 'https://nvd.nist.gov/vuln/search/results?form_type=Basic&results_type=overview&query='+search_term+'&search_type=all&startIndex='+str(page*20)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'lxml')    
    table = soup.find("table", {'class': 'table table-striped table-hover'})
    rows = table.findAll('tr')
    
    #print page we are on and the url for that page
    print("We are on Page #" + str(page) +"")
    print(url)  
    
    #in each row go through and add each element needed to the arrays to eventually display
    for tr in rows[1:]: #from 2nd row
        
        #find table header
        table_head = tr.find('th')
        
        #append row id to the id array
        vul_IDs_ = table_head.find(text = True)
        vul_IDs.append(vul_IDs_)
        
        #append the severity score to the severity array
        sev = tr.find("td", {'nowrap': 'nowrap'})
        sevLevel = sev.findAll('a')
        if sevLevel == []:
            severities.append(10.0)
        else:
            sevNumber = sevLevel[0].find(text = True)
            splitSev = sevNumber.split(" ")
            severities.append(float(splitSev[0]))
        
        ##get publish dates into the dates array
        publish_ = tr.find('span')
        publish_dates_ = publish_.find(text = True)
        publish_dates.append(publish_dates_)
        
        #get urls into the url array
        url = table_head.find('a')
        urls.append("https://nvd.nist.gov" + url['href'])

print("\n")

#number of records count
count = 1

#print records for the rows that match the criteria
for vul in vul_IDs:
    convert_date = dt.strptime(publish_dates[vul_IDs.index(vul)], "%B %d, %Y; %H:%M:%S %p %z")
    published_converted = dt.isoformat(convert_date)
    if (dt.isoformat(start_date) <= published_converted <= dt.isoformat(end_date)):
        if severities[vul_IDs.index(vul)] >= min_severity:
            print("No. " + str(count))
            print("Vul_Id: " + vul)
            print("Severity: " + str(severities[vul_IDs.index(vul)]))
            print("Publish Date: " + str(published_converted))                   
            print("For more information, visit " + urls[vul_IDs.index(vul)])
            print('------------------------------------------------------------------------------------')
            count += 1
        else:
            continue
    else:
        continue
        

Enter a search term:python
Type in a minimum severity level (1-10):8.5
Type in a start publish date (MM-DD-YYYY):10-13-2012
Type in a end publish date (MM-DD-YYYY):11-30-2015


The search returned 534 results. Use the following criteria to refine your search.
There are 26 pages 

Genterating Results, please wait...



We are on Page #0
https://nvd.nist.gov/vuln/search/results?form_type=Basic&results_type=overview&query=python&search_type=all&startIndex=0
We are on Page #1
https://nvd.nist.gov/vuln/search/results?form_type=Basic&results_type=overview&query=python&search_type=all&startIndex=20
We are on Page #2
https://nvd.nist.gov/vuln/search/results?form_type=Basic&results_type=overview&query=python&search_type=all&startIndex=40
We are on Page #3
https://nvd.nist.gov/vuln/search/results?form_type=Basic&results_type=overview&query=python&search_type=all&startIndex=60
We are on Page #4
https://nvd.nist.gov/vuln/search/results?form_type=Basic&results_type=overview&query=python&search_type